# Brain Tumor Patient Survival Prediction

#### Challenge is to develop a model to predict brain tumor patient survival. The goal is both to create a high-performing algorithm for the target task, as well as to analyze performance across several different architecture permutations. Three different network designs are designed. Each model is built and trained, and the final of each  model is saved as `*.hdf5` file.

### Jarvis library

In [1]:
% pip install jarvis-md

     |████████████████████████████████| 81kB 5.4MB/s 
     |████████████████████████████████| 645kB 40.7MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Imports


In [2]:
import os, numpy as np, pandas as pd
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers
from jarvis.train import datasets, custom

# Data

The data used here will consist of brain tumor MRI exams derived from the MICCAI Brain Tumor Segmentation Challenge (BRaTS).

In [3]:
datasets.download(name='mr/brats-2020-096')

[ 2021-06-06 20:08:27 ] [====================] 100.000% : Extracting archive (0000486 / 0000486) 

{'code': '/data/raw/mr_brats_2020', 'data': '/data/raw/mr_brats_2020'}

### Python generators

In [4]:
configs = {'specs': {'ys': {
    'tumor': {
        'dtype': 'uint8',
        'loads': 'lbl-crp',
        'norms': {'clip': {'min': 0, 'max': 1}},
        'shape': [96, 96, 96, 1]},
    'survival': {
        'dtype': 'float32',
        'loads': 'survival_days_norm',
        'shape': [1]}}}}

gen_train, gen_valid, client = datasets.prepare(
    name='mr/brats-2020-096', 
    keyword='096*vox-org',
    configs=configs)

In [5]:
xs, ys = next(gen_train)

print('xs keys: {}'.format(xs.keys()))
print('ys keys: {}'.format(ys.keys()))

print('xs shape: {}'.format(xs['dat'].shape))
print('ys shape: {}'.format(ys['tumor'].shape))
print('ys shape: {}'.format(ys['survival'].shape))

xs keys: dict_keys(['dat'])
ys keys: dict_keys(['tumor', 'survival'])
xs shape: (3, 96, 96, 96, 4)
ys shape: (3, 96, 96, 96, 1)
ys shape: (3, 1)


### Model inputs

In [6]:
inputs = client.get_inputs(Input)

print(inputs)
print(inputs.keys())
print(inputs['dat'].shape)

{'dat': <KerasTensor: shape=(None, 96, 96, 96, 4) dtype=float32 (created by layer 'dat')>}
dict_keys(['dat'])
(None, 96, 96, 96, 4)


# Training

### Model 1 - Fully Convolutional Network using Contracting and Expanding layers

In [ ]:
kwargs = {
    'kernel_size': (3, 3, 3),
    'padding': 'same'}

conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)
tran = lambda x, filters, strides : layers.Conv3DTranspose(filters=filters, strides=strides, **kwargs)(x)
concat = lambda a, b : layers.Concatenate()([a, b])

conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=2)))
tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=2)))

l1 = conv1(8, inputs['dat'])
l2 = conv1(16, conv2(16, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))
l5 = conv1(64, conv2(64, l4))

l6  = tran2(48, l5)
l7  = tran2(32, conv1(48, concat(l4, l6)))
l8  = tran2(16, conv1(32, concat(l3, l7)))
l9  = tran2(8,  conv1(16, concat(l2, l8)))
l10 = conv1(8,  l9)

logits = {}
logits['survival'] = layers.Conv3D(filters=1, name='survival', activation='sigmoid', **kwargs)(l10)
logits['tumor'] = layers.Conv3D(filters=2, name='tumor', **kwargs)(l10)
model1 = Model(inputs=inputs, outputs=logits)

In [ ]:
model1.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4),
    loss={
        'survival': losses.MeanSquaredError(),
        'tumor': losses.SparseCategoricalCrossentropy(from_logits=True)},
    experimental_run_tf_function=False)

In [ ]:
client.load_data_in_memory()

[ 2021-06-06 15:03:04 ] [====================] 100.000% : Iterating | 000235    

In [ ]:
def generator(G):
    for xs, ys in G:
 
        survival = ys['survival'].reshape(-1, 1, 1, 1, 1)
 
        ys['survival'] = np.zeros((survival.shape[0], 96, 96, 96, 1), dtype='float32')
        ys['survival'][:] = survival
 
        yield xs, ys

In [ ]:
model1.fit(
    x=generator(gen_train), 
    steps_per_epoch=125, 
    epochs=4,
    validation_data=generator(gen_valid),
    validation_steps=125,
    validation_freq=2,
    use_multiprocessing=True)

Epoch 1/4
125/125 [==============================] - 205s 1s/step - loss: 0.3026 - survival_loss: 0.0321 - tumor_loss: 0.2706
Epoch 2/4
125/125 [==============================] - 207s 2s/step - loss: 0.1310 - survival_loss: 0.0117 - tumor_loss: 0.1193 - val_loss: 0.1336 - val_survival_loss: 0.0079 - val_tumor_loss: 0.1257
Epoch 3/4
125/125 [==============================] - 165s 1s/step - loss: 0.1105 - survival_loss: 0.0111 - tumor_loss: 0.0994
Epoch 4/4
125/125 [==============================] - 206s 2s/step - loss: 0.0981 - survival_loss: 0.0110 - tumor_loss: 0.0871 - val_loss: 0.1096 - val_survival_loss: 0.0070 - val_tumor_loss: 0.1026


In [ ]:
test_train, test_valid = client.create_generators(test=True)

In [ ]:
preds_train = []
trues_train = []
mae_train = []

for x, y in test_train:
    
    logits = model1.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['survival']

    preds_train.append(logits.ravel())
    trues_train.append(y['survival'].ravel())
    mae_train.append(np.abs(preds_train[-1] - trues_train[-1]))

mae_train = np.array(mae_train).ravel()

[ 2021-06-06 15:17:24 ] [====================] 100.000% : Iterating | 000185    

In [ ]:
df1_train = pd.DataFrame(index=np.arange(mae_train.size))
df1_train['MAE'] = mae_train

df1_train_mean = df1_train['MAE'].mean()
df1_train_median = df1_train['MAE'].median()
df1_train_25per = df1_train['MAE'].quantile(0.25)
df1_train_75per = df1_train['MAE'].quantile(0.75)

In [ ]:
preds_valid = []
trues_valid = []
mae_valid = []

for x, y in test_valid:
    
    logits = model1.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['survival']

    preds_valid.append(logits.ravel())
    trues_valid.append(y['survival'].ravel())
    mae_valid.append(np.abs(preds_valid[-1] - trues_valid[-1]))

mae_valid = np.array(mae_valid).ravel()

[ 2021-06-06 15:18:00 ] [====================] 100.000% : Iterating | 000050    

In [ ]:
df1_valid = pd.DataFrame(index=np.arange(mae_valid.size))
df1_valid['MAE'] = mae_valid

df1_valid_mean = df1_valid['MAE'].mean()
df1_valid_median = df1_valid['MAE'].median()
df1_valid_25per = df1_valid['MAE'].quantile(0.25)
df1_valid_75per = df1_valid['MAE'].quantile(0.75)

In [ ]:
print(df1_train_median)
print(df1_valid_median)

0.06330692768096924
0.0630541443824768


In [ ]:
model1.save('./Model1.hdf5')

### Model 2 - Global regression type loss function using Dense Layers

In [7]:
kwargs = {
    'kernel_size': (3, 3, 3),
    'padding': 'same'}

conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)
tran = lambda x, filters, strides : layers.Conv3DTranspose(filters=filters, strides=strides, **kwargs)(x)
concat = lambda a, b : layers.Concatenate()([a, b])

conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=2)))
tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=2)))

l1 = conv1(16, inputs['dat'])
l2 = conv1(24, conv2(24, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))
l5 = conv1(64, conv2(64, l4))
l6  = tran2(48, l5)
l7  = tran2(32, conv1(48, concat(l4, l6)))
l8  = tran2(16, conv1(32, concat(l3, l7)))
l9  = tran2(8,  conv1(16, concat(l2, l8)))
l10 = conv1(8,  l9)

logits = {}
logits['survival'] = layers.Dense(1, activation='sigmoid', name='survival')(l10)
logits['tumor'] = layers.Dense(1, activation='sigmoid', name='tumor')(l10)
model2 = Model(inputs=inputs, outputs=logits)

In [8]:
model2.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4),
    loss={
        'survival': losses.MeanSquaredError(),
        'tumor': losses.MeanSquaredError()},
    experimental_run_tf_function=False)

In [9]:
def generator(G):
    for xs, ys in G:
 
        survival = ys['survival'].reshape(-1, 1, 1, 1, 1)
 
        ys['survival'] = np.zeros((survival.shape[0], 96, 96, 96, 1), dtype='float32')
        ys['survival'][:] = survival
 
        yield xs, ys

In [10]:
model2.fit(
    x=generator(gen_train), 
    steps_per_epoch=125, 
    epochs=4,
    validation_data=generator(gen_valid),
    validation_steps=125,
    validation_freq=2,
    use_multiprocessing=True)

Epoch 1/4
125/125 [==============================] - 100s 499ms/step - loss: 0.1443 - survival_loss: 0.0281 - tumor_loss: 0.1162
Epoch 2/4
125/125 [==============================] - 146s 1s/step - loss: 0.1008 - survival_loss: 0.0199 - tumor_loss: 0.0810 - val_loss: 0.0931 - val_survival_loss: 0.0154 - val_tumor_loss: 0.0777
Epoch 3/4
125/125 [==============================] - 64s 508ms/step - loss: 0.0845 - survival_loss: 0.0164 - tumor_loss: 0.0681
Epoch 4/4
125/125 [==============================] - 117s 936ms/step - loss: 0.0742 - survival_loss: 0.0153 - tumor_loss: 0.0589 - val_loss: 0.0713 - val_survival_loss: 0.0152 - val_tumor_loss: 0.0561


In [11]:
test_train, test_valid = client.create_generators(test=True)

In [12]:
preds_train = []
trues_train = []
mae_train = []

for x, y in test_train:
    
    logits = model2.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['survival']

    preds_train.append(logits.ravel())
    trues_train.append(y['survival'].ravel())
    mae_train.append(np.abs(preds_train[-1] - trues_train[-1]))

mae_train = np.array(mae_train).ravel()

[ 2021-06-06 20:17:00 ] [====================] 100.000% : Iterating | 000185    

In [13]:
df2_train = pd.DataFrame(index=np.arange(mae_train.size))
df2_train['MAE'] = mae_train

df2_train_mean = df2_train['MAE'].mean()
df2_train_median = df2_train['MAE'].median()
df2_train_25per = df2_train['MAE'].quantile(0.25)
df2_train_75per = df2_train['MAE'].quantile(0.75)

In [18]:
preds_valid = []
trues_valid = []
mae_valid = []

for x, y in test_valid:
    
    logits = model2.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['survival']

    preds_valid.append(logits.ravel())
    trues_valid.append(y['survival'].ravel())
    mae_valid.append(np.abs(preds_valid[-1] - trues_valid[-1]))

mae_valid = np.array(mae_valid).ravel()

[ 2021-06-06 20:17:31 ] [====================] 100.000% : Iterating | 000050    

In [19]:
df2_valid = pd.DataFrame(index=np.arange(mae_valid.size))
df2_valid['MAE'] = mae_valid

df2_valid_mean = df2_valid['MAE'].mean()
df2_valid_median = df2_valid['MAE'].median()
df2_valid_25per = df2_valid['MAE'].quantile(0.25)
df2_valid_75per = df2_valid['MAE'].quantile(0.75)

In [21]:
Data = {"": ["model2"],
        "Mean Train":[df2_train_mean],
        "Median Train": [df2_train_median],
        "25th Tile Train": [df2_train_25per],
        "75th Tile Train": [df2_train_75per],
        "Mean Valid":[df2_valid_mean],
        "Median Valid": [df2_valid_median],
        "25th Tile Valid": [df2_valid_25per],
        "75th Tile Valid": [df2_valid_75per]}

df = pd.DataFrame(Data)           
fname = './bmuthuma_results2.csv'
df.to_csv(fname)

In [22]:
model2.save('./Model2.hdf5')

### Model 3 - Pretrained Auto Encoder Strategy using SE Net

In [7]:
kwargs = {
    'kernel_size': (3, 3, 3),
    'padding': 'same'}

conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)
tran = lambda x, filters, strides : layers.Conv3DTranspose(filters=filters, strides=strides, **kwargs)(x)
concat = lambda a, b : layers.Concatenate()([a, b])

conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=2)))
tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=2)))

l1 = conv1(8, inputs['dat'])
l2 = conv1(16, conv2(16, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))
l5 = conv1(64, conv2(64, l4))

p1 = layers.GlobalAveragePooling3D()(l5)

ch = int(p1.shape[-1] / 4)
f1 = layers.Dense(ch, activation='relu')(p1)

scale = layers.Dense(l5.shape[-1], activation='sigmoid')(f1)
scale = layers.Reshape((1, 1, 1, l5.shape[-1]))(scale)    

l5 = l5 * scale

n0, n1, c = l5.shape[-3:]
f0 = layers.Reshape([-1, 1, 1, n0 * n1 * c])(l5)

l6 = tran2(48, l5)

concat = lambda a, b : layers.Concatenate()([a, b])
concat(l4, l6)

l7 = tran2(32, conv1(48, l6 + conv1(48, l4)))
l8 = tran2(16, conv1(32, l7 + conv1(32, l3)))
l9 = tran2(8,  conv1(16, l8 + conv1(16, l2)))
l10 = conv1(8,  l9)

encoder = Model(inputs=inputs, outputs=l10)

In [8]:
encoder.trainable = False
latent = encoder(inputs)
h0 = layers.Flatten()(latent)
h1 = layers.Dense(32, activation='relu')(h0)

logits = {}
logits['survival'] = layers.Dense(1, activation='sigmoid', name='survival')(latent)
logits['tumor'] = layers.Dense(1, activation='sigmoid', name='tumor')(latent)
model3 = Model(inputs=inputs, outputs=logits)

In [9]:
model3.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4),
    loss={
        'survival': losses.MeanSquaredError(),
        'tumor': losses.MeanSquaredError()},
    experimental_run_tf_function=False)

In [10]:
def generator(G):
    for xs, ys in G:
 
        survival = ys['survival'].reshape(-1, 1, 1, 1, 1)
 
        ys['survival'] = np.zeros((survival.shape[0], 96, 96, 96, 1), dtype='float32')
        ys['survival'][:] = survival
 
        yield xs, ys

In [11]:
model3.fit(
    x=generator(gen_train), 
    steps_per_epoch=125, 
    epochs=4,
    validation_data=generator(gen_valid),
    validation_steps=125,
    validation_freq=2,
    use_multiprocessing=True)

Epoch 1/4
125/125 [==============================] - 88s 409ms/step - loss: 0.2565 - survival_loss: 0.0151 - tumor_loss: 0.2414
Epoch 2/4
125/125 [==============================] - 135s 1s/step - loss: 0.2514 - survival_loss: 0.0139 - tumor_loss: 0.2375 - val_loss: 0.2509 - val_survival_loss: 0.0141 - val_tumor_loss: 0.2368
Epoch 3/4
125/125 [==============================] - 47s 379ms/step - loss: 0.2478 - survival_loss: 0.0138 - tumor_loss: 0.2340
Epoch 4/4
125/125 [==============================] - 104s 834ms/step - loss: 0.2435 - survival_loss: 0.0128 - tumor_loss: 0.2307 - val_loss: 0.2426 - val_survival_loss: 0.0122 - val_tumor_loss: 0.2305


In [12]:
test_train, test_valid = client.create_generators(test=True)

In [13]:
preds_train = []
trues_train = []
mae_train = []

for x, y in test_train:
    
    logits = model3.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['survival']

    preds_train.append(logits.ravel())
    trues_train.append(y['survival'].ravel())
    mae_train.append(np.abs(preds_train[-1] - trues_train[-1]))

mae_train = np.array(mae_train).ravel()

[ 2021-06-06 20:03:59 ] [====================] 100.000% : Iterating | 000185    

In [14]:
df3_train = pd.DataFrame(index=np.arange(mae_train.size))
df3_train['MAE'] = mae_train

df3_train_mean = df3_train['MAE'].mean()
df3_train_median = df3_train['MAE'].median()
df3_train_25per = df3_train['MAE'].quantile(0.25)
df3_train_75per = df3_train['MAE'].quantile(0.75)

In [16]:
preds_valid = []
trues_valid = []
mae_valid = []

for x, y in test_valid:
    logits = model3.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['survival']

    preds_valid.append(logits.ravel())
    trues_valid.append(y['survival'].ravel())
    mae_valid.append(np.abs(preds_valid[-1] - trues_valid[-1]))

mae_valid = np.array(mae_valid).ravel()

[ 2021-06-06 20:04:45 ] [====================] 100.000% : Iterating | 000050    

In [17]:
df3_valid = pd.DataFrame(index=np.arange(mae_valid.size))
df3_valid['MAE'] = mae_valid

df3_valid_mean = df3_valid['MAE'].mean()
df3_valid_median = df3_valid['MAE'].median()
df3_valid_25per = df3_valid['MAE'].quantile(0.25)
df3_valid_75per = df3_valid['MAE'].quantile(0.75)

In [21]:
model3.save('./Model3.hdf5')

# Evaluation

For each of the three models, the following metrics is calculated for **both the training and validation** cohorts:

* absolute error, mean
* absolute error, median
* absolute error, 25th percentile
* absolute error, 75th percentile 

In [ ]:
Data = {"": ["model1", "model2", "model3"],
        "Mean Train":[df1_train_mean, df2_train_mean, df3_train_mean],
        "Median Train": [df1_train_median, df2_train_median, df3_train_median],
        "25th Tile Train": [df1_train_25per, df2_train_25per, df3_train_25per],
        "75th Tile Train": [df1_train_75per, df2_train_75per, df3_train_75per],
        "Mean Valid":[df1_valid_mean, df2_valid_mean, df3_valid_mean],
        "Median Valid": [df1_valid_median, df2_valid_median, df3_valid_median],
        "25th Tile Valid": [df1_valid_25per, df2_valid_25per, df3_valid_25per],
        "75th Tile Valid": [df1_valid_75per, df2_valid_75per, df3_valid_75per]}

### Results

Stored **training and validation** cohort statistics for the different models in a csv file.

In [ ]:
df = pd.DataFrame(Data)           
fname = './Results.csv'
df.to_csv(fname)